# Converting other formats to correctionlib json

An attempt to convert all the correction types known to CMS coffea `lookup_tools`.

In [1]:
import uproot4

examples = "https://raw.githubusercontent.com/CoffeaTeam/coffea/master/tests/samples"
sf = uproot4.open(f"{examples}/testSF2d.histo.root:scalefactors_Tight_Electron").to_boost()

`sf` is a binned event weight to be applied as a function of $\eta$ and $p_T$

In [2]:
sf

Histogram(
  Variable([-2.5, -2, -1.566, -1.444, -0.8, 0, 0.8, 1.444, 1.566, 2, 2.5]),
  Variable([10, 20, 35, 50, 90, 150, 500]),
  storage=Weight()) # Sum: WeightedSum(value=58.2612, variance=0.0410892) (WeightedSum(value=68.4267, variance=0.0780986) with flow)

In [3]:
from correctionlib.schemav1 import Correction

corr1 = Correction.parse_obj(
    {
        "version": 0,
        "name": "scalefactors_Tight_Electron",
        "inputs": [
            {"type": "real", "name": "eta", "description": "possibly supercluster eta?"},
            {"name": "pt", "type": "real"},
        ],
        "output": {"name": "weight", "type": "real"},
        "data": {
            "nodetype": "multibinning",
            "edges": [
                list(sf.axes[0].edges),
                list(sf.axes[1].edges),
            ],
            "content": list(sf.view().value.flatten()),
        },
    }
)

In [4]:
corr1

Correction(name='scalefactors_Tight_Electron', description=None, version=0, inputs=[Variable(name='eta', type='real', description='possibly supercluster eta?'), Variable(name='pt', type='real', description=None)], output=Variable(name='weight', type='real', description=None), data=MultiBinning(nodetype='multibinning', edges=[[-2.5, -2.0, -1.566, -1.444, -0.8, 0.0, 0.8, 1.444, 1.566, 2.0, 2.5], [10.0, 20.0, 35.0, 50.0, 90.0, 150.0, 500.0]], content=[0.8065573573112488, 0.8824593424797058, 0.9188405871391296, 0.940397322177887, 1.0510855913162231, 1.0510855913162231, 0.8285714387893677, 0.9274873733520508, 0.9669876098632812, 0.9809644818305969, 1.0059242248535156, 1.0059242248535156, 1.0328638553619385, 1.0075949430465698, 0.9881955981254578, 0.9953917264938354, 1.1041055917739868, 1.1041055917739868, 1.0077519416809082, 0.9720394611358643, 0.9746666550636292, 0.9716748595237732, 0.9894859790802002, 0.9894859790802002, 0.9407216310501099, 0.9529983997344971, 0.9534574747085571, 0.953086

In [5]:
import pandas

sf = pandas.read_csv(f"{examples}/DeepCSV_2016LegacySF_V1_TuneCP5.btag.csv.gz", skipinitialspace=True)
sf

,DeepCSV;OperatingPoint,measurementType,sysType,jetFlavor,etaMin,etaMax,ptMin,ptMax,discrMin,discrMax,formula
0,3,iterativefit,central,1,0.0,2.4,20.0,10000.0,-15.0000,1.1000,1.0
1,3,iterativefit,down_hfstats2,0,0.0,2.5,20.0,30.0,-15.0000,0.0000,0.875507235527
2,3,iterativefit,down_hfstats2,0,0.0,2.5,20.0,30.0,0.0000,0.1060,1.12514662743
3,3,iterativefit,down_hfstats2,0,0.0,2.5,20.0,30.0,0.1060,0.2455,80.6915943226*(x-0.106)*(x-0.106)*(x-0.106)-33...
4,3,iterativefit,down_hfstats2,0,0.0,2.5,20.0,30.0,0.2455,0.3085,2286.53836199*(x-0.2455)*(x-0.2455)*(x-0.2455)...
...,...,...,...,...,...,...,...,...,...,...,...
17642,3,iterativefit,up_jesPileUpPtBB,2,1.6,2.5,60.0,10000.0,0.2690,0.3300,220.170853445*(x-0.269)*(x-0.269)*(x-0.269)-26...
17643,3,iterativefit,up_jesPileUpPtBB,2,1.6,2.5,60.0,10000.0,0.3300,0.4045,123.106230417*(x-0.33)*(x-0.33)*(x-0.33)-8.720...
17644,3,iterativefit,up_jesPileUpPtBB,2,1.6,2.5,60.0,10000.0,0.4045,0.5030,-36.1901842061*(x-0.4045)*(x-0.4045)*(x-0.4045...
17645,3,iterativefit,up_jesPileUpPtBB,2,1.6,2.5,60.0,10000.0,0.5030,0.7850,-0.610093327584*(x-0.503)*(x-0.503)*(x-0.503)-...


In [6]:
from correctionlib.schemav1 import Binning, Category, Formula


def build_formula(sf):
    if len(sf) != 1:
        raise ValueError(sf)
    
    value = sf.iloc[0]["formula"]
    if "x" in value:
        return Formula.parse_obj({
            "expression": value,
            "parser": "TFormula",
            # For this case, since this is a "reshape" SF, we know the parameter is the discriminant
            "parameters": [4],
        })
    else:
        return float(value)

def build_discrbinning(sf):
    edges = sorted(set(sf['discrMin']) | set(sf['discrMax']))
    return Binning.parse_obj({
        "nodetype": "binning",
        "edges": edges,
        "content": [
            build_formula(sf[(sf['discrMin'] >= lo) & (sf['discrMax'] <= hi)])
            for lo, hi in zip(edges[:-1], edges[1:])
        ]
    })

def build_ptbinning(sf):
    edges = sorted(set(sf['ptMin']) | set(sf['ptMax']))
    return Binning.parse_obj({
        "nodetype": "binning",
        "edges": edges,
        "content": [
            build_discrbinning(sf[(sf['ptMin'] >= lo) & (sf['ptMax'] <= hi)])
            for lo, hi in zip(edges[:-1], edges[1:])
        ]
    })

def build_etabinning(sf):
    edges = sorted(set(sf['etaMin']) | set(sf['etaMax']))
    return Binning.parse_obj({
        "nodetype": "binning",
        "edges": edges,
        "content": [
            build_ptbinning(sf[(sf['etaMin'] >= lo) & (sf['etaMax'] <= hi)])
            for lo, hi in zip(edges[:-1], edges[1:])
        ]
    })

def build_flavor(sf):
    keys = sorted(sf['jetFlavor'].unique())
    return Category.parse_obj({
        "nodetype": "category",
        "keys": keys,
        "content": [
            build_etabinning(sf[sf['jetFlavor'] == key])
            for key in keys
        ]
    })

def build_systs(sf):
    keys = list(sf['sysType'].unique())
    return Category.parse_obj({
        "nodetype": "category",
        "keys": keys,
        "content": [
            build_flavor(sf[sf['sysType'] == key])
            for key in keys
        ]
    })


corr2 = Correction.parse_obj(
    {
        "version": 1,
        "name": "DeepCSV_2016LegacySF",
        "description": "A btagging scale factor",
        "inputs": [
            {"name": "systematic", "type": "string"},
            {"name": "flavor", "type": "int", "description": "BTV flavor definiton: 0=b, 1=c, 2=udsg"},
            {"name": "abseta", "type": "real"},
            {"name": "pt", "type": "real"},
            {"name": "discriminant", "type": "real", "description": "DeepCSV output value"},
        ],
        "output": {"name": "weight", "type": "real"},
        "data": build_systs(sf),
    }
)

In [7]:
import requests

sf = requests.get(f"{examples}/EIDISO_WH_out.histo.json").json()

In [8]:
def build_syst(sf):
    return Category.parse_obj({
        "nodetype": "category",
        "keys": ["nominal", "up", "down"],
        "content": [
            sf["value"],
            sf["value"] + sf["error"],
            sf["value"] - sf["error"],
        ]
    })


def parse_str(key, prefix="eta:"):
    if not key.startswith(prefix + "["):
        raise ValueError(f"{key} missing prefix {prefix}")
    lo, hi = map(float, key[len(prefix + "["):-1].split(","))
    return lo, hi


def build_pts(sf):
    edges = []
    content = []
    for binstr, data in sf.items():
        if not binstr.startswith("pt:["):
            raise ValueError
        lo, hi = map(float, binstr[len("pt:["):-1].split(","))
        if len(edges) == 0:
            edges.append(lo)
        if edges[-1] != lo:
            raise ValueError
        edges.append(hi)
        content.append(build_syst(data))
    
    return Binning.parse_obj({
        "nodetype": "binning",
        "edges": edges,
        "content": content,
    })


def build_etas(sf):
    bins = [parse_str(s, "eta:") for s in sf]
    edges = sorted(set(edge for bin in bins for edge in bin))
    content = [None] * (len(edges) - 1)
    for s, data in sf.items():
        lo, hi = parse_str(s, "eta:")
        found = False
        for i, bin in enumerate(bins):
            if bin[0] >= lo and bin[1] <= hi:
                content[i] = build_pts(data)
                found = True

        
    return Binning.parse_obj({
        "nodetype": "binning",
        "edges": edges,
        "content": content,
    })


corr3 = Correction.parse_obj(
    {
        "version": 1,
        "name": "EIDISO_WH_out",
        "description": "An electron scale factor",
        "inputs": [
            {"name": "eta", "type": "real"},
            {"name": "pt", "type": "real"},
            {"name": "systematic", "type": "string"},
        ],
        "output": {"name": "weight", "type": "real"},
        "data": build_etas(sf["EIDISO_WH"]["eta_pt_ratio"]),
    }
)

In [9]:
from correctionlib.schemav1 import CorrectionSet
import gzip

cset = CorrectionSet.parse_obj({
    "schema_version": 1,
    "corrections": [
        corr1,
        corr2,
        corr3,
    ]
})

with open("data/examples.json", "w") as fout:
    fout.write(cset.json(exclude_unset=True, indent=4))

with gzip.open("data/examples.json.gz", "wt") as fout:
    fout.write(cset.json(exclude_unset=True))